In [1]:
import zipfile
import urllib2
import os

In [4]:
s_url = 'ftp://ftp.nhtsa.dot.gov/GES/GES12/GES12_Flatfile.zip'
zip_name = 'GES12_Flatfile.zip'
cwd = os.getcwd()
dir_path = os.path.join(cwd, 'GES12')
zip_path = os.path.join(dir_path, zip_name)

In [7]:
if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [ ]:
if not os.path.exists(zip_path):
    response = urllib2.urlopen(s_url)
    with open(zip_path,'wb') as f:
        x = response.read()
        f.write(x)    

In [10]:
with zipfile.ZipFile(os.path.join(dir_path, zip_path),'r') as z:
    z.extractall(dir_path)

In [12]:
import pandas as pd
import numpy as np
import sklearn

In [13]:
input_file_path = os.path.join(dir_path, 'PERSON.txt')

In [23]:
input_data = pd.read_csv(input_file_path, delimiter = '\t')

In [31]:
input_data.head()

,CASENUM,VEH_NO,PER_NO,SEX_IM,INJSEV_IM,EJECT_IM,PERALCH_IM,SEAT_IM,AGE_IM,VE_FORMS,WEIGHT,REGION,STRATUM,PJ,PSU,PSUSTRAT,STR_VEH,MONTH,HOUR,MINUTE,HARM_EV,MAN_COLL,SCH_BUS,MAKE,BODY_TYP,MOD_YEAR,TOW_VEH,SPEC_USE,EMER_USE,ROLLOVER,IMPACT1,FIRE_EXP,AGE,SEX,PER_TYP,INJ_SEV,SEAT_POS,REST_USE,REST_MIS,AIR_BAG,EJECTION,DRINKING,ALC_STATUS,ATST_TYP,ALC_RES,DRUGS,DSTATUS,DRUGTST1,DRUGTST2,DRUGTST3,DRUGRES1,DRUGRES2,DRUGRES3,HOSPITAL,P_SF1,P_SF2,P_SF3,LOCATION
0,201210432623,1,1,1,0,0,0,11,43,2,27.146,1,2,3,2,3,0,1,14,7,12,6,0,23.0,64.0,2007.0,0.0,0.0,0.0,0.0,12.0,0.0,43,1,1,0,11,3,0,98,0,0,8,95,95,0,8,6,0,0,95,0,0,0,0,0,0,0
1,201210432623,2,1,2,0,0,0,11,68,2,27.146,1,2,3,2,3,0,1,14,7,12,6,0,6.0,29.0,2006.0,0.0,0.0,0.0,0.0,62.0,0.0,68,2,1,0,11,3,0,98,0,0,8,95,95,0,8,6,0,0,95,0,0,0,0,0,0,0
2,201210432628,1,1,1,1,0,0,11,67,2,54.292,1,5,3,2,3,0,1,13,41,12,1,0,20.0,49.0,2005.0,0.0,0.0,0.0,0.0,6.0,0.0,67,1,1,1,11,3,0,98,0,0,8,95,95,0,8,6,0,0,95,0,0,5,0,0,0,0
3,201210432628,1,2,2,1,0,0,13,69,2,54.292,1,5,3,2,3,0,1,13,41,12,1,0,20.0,49.0,2005.0,0.0,0.0,0.0,0.0,6.0,0.0,69,2,2,1,13,3,0,98,0,8,0,0,96,8,0,0,0,0,0,0,0,5,0,0,0,0
4,201210432628,2,1,2,0,0,0,11,36,2,54.292,1,5,3,2,3,0,1,13,41,12,1,0,12.0,39.0,2000.0,0.0,0.0,0.0,0.0,12.0,0.0,36,2,1,0,11,3,0,98,0,0,8,95,95,0,8,6,0,0,95,0,0,0,0,0,0,0


In [24]:
input_data['INJSEV_IM'].value_counts()

0    100840
2     20758
1     19380
3      9738
5      1179
4      1178
6         4
Name: INJSEV_IM, dtype: int64

In [25]:
input_data = input_data[input_data.INJSEV_IM != 6]

In [27]:
for cols in input_data.columns:
    n_nan = input_data[cols].isnull().sum()
    if n_nan > 0:
        print cols, n_nan

MAKE 5162
BODY_TYP 5162
MOD_YEAR 5162
TOW_VEH 5162
SPEC_USE 5162
EMER_USE 5162
ROLLOVER 5162
IMPACT1 5162
FIRE_EXP 5162


In [30]:
pd.set_option('display.max.columns',100)

In [32]:
print input_data.shape

(153073, 58)


In [33]:
data = input_data[~input_data.MAKE.isnull()]

In [35]:
discarded = data.pop('INJ_SEV')

In [36]:
target = data.pop('INJSEV_IM')

In [37]:
print data.shape

(147911, 56)


In [43]:
target.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: INJSEV_IM, dtype: float64

In [42]:
target = (target == 4).astype('float')

In [60]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score
from sklearn.grid_search import GridSearchCV

In [53]:
xtrain, xtest, ytrain, ytest = sklearn.cross_validation.train_test_split(data, target, train_size = 0.5)

In [61]:
linreg = LinearRegression()
linreg.fit(xtrain, ytrain)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [62]:
lr_preds = linreg.predict(xtest)
lr_perf = roc_auc_score(ytest, lr_preds)
print 'OLS: Area Under the ROC Curve = {}'.format(lr_perf)

OLS: Area Under the ROC Curve = 0.936801085828


In [63]:
from sklearn.linear_model import Ridge

In [64]:
ridge = GridSearchCV(Ridge(), {'alpha':np.logspace(-10,10,10)})
ridge_model = ridge.fit(xtrain, ytrain)
ri_preds = ridge_model.predict(xtest)
ri_perf = roc_auc_score(ytest, ri_preds)
print 'Ridge : Area Under the ROC Curve = {}'.format(ri_perf)

Ridge : Area Under the ROC Curve = 0.93680132195


In [66]:
from sklearn.linear_model import Lasso

In [70]:
lasso = GridSearchCV(Lasso(max_iter = 1500), {'alpha':np.logspace(-10,-8,5)})
lasso_model = lasso.fit(xtrain, ytrain)
la_preds = lasso_model.predict(xtest)
la_perf = roc_auc_score(ytest, la_preds)
print 'Lasso : Area Under the ROC Curve = {}'.format(la_perf)

Lasso : Area Under the ROC Curve = 0.936801474735


In [71]:
from sklearn.ensemble import GradientBoostingClassifier

In [76]:
gbm = GradientBoostingClassifier(n_estimators = 500)

In [77]:
gbm.fit(xtrain, ytrain)
gbm_preds = gbm.predict_proba(xtest)[:,1]
gbm_perf = roc_auc_score(ytest, gbm_preds)
print 'GBM : Area Under the ROC Curve = {}'.format(gbm_perf)

GBM : Area Under the ROC Curve = 0.975948696523


In [81]:
from sklearn.tree import DecisionTreeClassifier

In [84]:
tree = GridSearchCV(DecisionTreeClassifier(), {'max_depth':np.arange(3,10)})
tree.fit(xtrain, ytrain)
tree_preds = tree.predict_proba(xtest)[:,1]
tree_perf = roc_auc_score(ytest, tree_preds)
print 'Decision Tree: Area Under the ROC Curve = {}'.format(tree_perf)

Decision Tree: Area Under the ROC Curve = 0.906524814755


In [91]:
importances = pd.Series(gbm.feature_importances_, index = data.columns)
print importances.order(ascending = False)[:10]

STRATUM     0.093225
WEIGHT      0.063758
EJECT_IM    0.054859
HARM_EV     0.045015
MOD_YEAR    0.044824
HOSPITAL    0.043615
DRINKING    0.043468
MINUTE      0.041991
AGE_IM      0.041362
HOUR        0.039497
dtype: float64


d:\anaconda\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: order is deprecated, use sort_values(...)
  from ipykernel import kernelapp as app
